In [3]:
!pip install datasets torch
!pip install evaluate

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [4]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline)

import pandas as pd
import numpy as np
import evaluate

In [5]:
dataset = load_dataset('sepidmnorozy/Korean_sentiment')
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/36000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1333 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2667 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 36000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1333
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 2667
    })
})

In [6]:
print(dataset['train'][3118])
print(dataset['test'][214])

{'label': 1, 'text': '졸잼!!!성아가나중에억울한일이잇어서좀슬펏는데마지막은기쁘게끝나서다행이에여'}
{'label': 0, 'text': '이 영화를 끝까지 볼려고 몇번이나 디비디도 빌려보고 다운로드도 받았는데 지금까지 메이저 영화중에 끝까지 다 보질 못한 영화중에 비트,졸리 나오는 스미스 부부 이후로 처음이다. 지루하고 하품나는 구성 왜 이 영화가 정이 안가나 했더니 문제는 올리버 스톤'}


## 토큰화 Tokenize

In [7]:
model_name = 'kykim/bert-kor-base'

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer

tokenizer_config.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/344k [00:00<?, ?B/s]

BertTokenizerFast(name_or_path='kykim/bert-kor-base', vocab_size=42000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [8]:
def tokenizer_func(x):
    return tokenizer(x['text'],
                     padding = 'max_length',
                     max_length = 256,
                     truncation = True)

In [9]:
tokenized_datasets = dataset.map(tokenizer_func, batched = True)

Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1333 [00:00<?, ? examples/s]

Map:   0%|          | 0/2667 [00:00<?, ? examples/s]

In [10]:
train_sample = 10000

train_ds = tokenized_datasets['train'].shuffle(seed=42).select(range(train_sample))
val_ds = tokenized_datasets['validation'].shuffle(seed=42)

전이학습 Transfer Learning

In [11]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 2)

pytorch_model.bin:   0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at kykim/bert-kor-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


하이퍼파라미터

In [12]:
ds = 32
epochs = 4
lr = 1e-5

In [13]:
args = TrainingArguments(
    'outputs',
    learning_rate = lr,
    warmup_ratio = 0.1,
    lr_scheduler_type = 'cosine',
    fp16 = True,
    evaluation_strategy='epoch',
    per_device_train_batch_size = ds,
    per_device_eval_batch_size = ds,
    gradient_accumulation_steps=4, #until bs =128
    eval_accumulation_steps = 4,
    num_train_epochs=epochs,
    weight_decay=0.01,
    report_to = 'none'
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Metrics

In [14]:
metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [18]:
class CustomTrainer(Trainer):
    def _save_checkpoint(self, model, trial, metrics=None):
        # 모델을 저장하기 전에 모든 텐서를 contiguous로 만듦
        for name, param in model.named_parameters():
            if param is not None:
                param.data = param.data.contiguous()
                if param.grad is not None:
                    param.grad.data = param.grad.data.contiguous()
        super()._save_checkpoint(model, trial, metrics)


모델 훈련

In [19]:
trainer = CustomTrainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
0,No log,0.311902,0.888972
1,No log,0.315423,0.886722
2,No log,0.317998,0.888972
3,No log,0.318501,0.890473


TrainOutput(global_step=312, training_loss=0.183225277142647, metrics={'train_runtime': 264.7943, 'train_samples_per_second': 151.061, 'train_steps_per_second': 1.178, 'total_flos': 5247486888099840.0, 'train_loss': 0.183225277142647, 'epoch': 3.987220447284345})

In [21]:
trainer.save_model('/content/drive/MyDrive/Final_project/NLP/sentiment_model')

In [22]:
pipe = pipeline('text-classification', model='/content/drive/MyDrive/Final_project/NLP/sentiment_model')

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


전이학습된 모델로 테스트셋 사용

In [23]:
test_data = dataset['validation'].shuffle(seed=424)[:100]
td = pd.DataFrame(test_data)
td

,label,text
0,1,"비밀은 적정선에 관한 이야기이기도 하다. 사람에게 돈이란, 평판이란, 책임이란, 기..."
1,0,메인 시나리오 뼈대가 없고 보여주기라기엔 용과의 전투씬도 너무 부족함..더욱이 결말...
2,1,올??? 찌질한 스커드가 노만 리더스 행님이었다니 !! ㅋㅋ
3,0,5점이 적당하나 평점이 쓸데없이 높아서 낮출 필요가 있슴...
4,0,"연출, 기획, 스토리 전부다 쓰레기..제인생 영화중에 제일 쓰레기.. 절대보지마요진심"
...,...,...
95,1,"""""""철아 엄마가 꼭 니 신세 갚고 죽을께"""""""
96,0,이건 뭐 .. 완전 삼류네
97,0,으... 밑도 끝도 없음. 여자가 처음부터 자기혈청을 제공했으면 카오스가 안왔을거 아님?
98,0,요즘에 보기에는 상당히 지루한 진부한 스릴러. 원초적 본능을 뛰어 넘기에도 힘든 영...


In [24]:
preds = pipe(td['text'].tolist())

preds_df = pd.DataFrame(preds)
preds_df

,label,score
0,LABEL_1,0.993731
1,LABEL_0,0.997724
2,LABEL_0,0.947795
3,LABEL_1,0.933018
4,LABEL_0,0.997567
...,...,...
95,LABEL_1,0.846441
96,LABEL_0,0.997439
97,LABEL_0,0.983951
98,LABEL_0,0.997635


In [25]:
preds_df.rename(columns={'label' : 'pred_label'}, inplace=True)
preds_df['pred_label'] = preds_df['pred_label'].map({'LABEL_0' : 0, 'LABEL_1' : 1})

preds_df = pd.concat([preds_df, td], axis=1)

In [26]:
preds_df

,pred_label,score,label,text
0,1,0.993731,1,"비밀은 적정선에 관한 이야기이기도 하다. 사람에게 돈이란, 평판이란, 책임이란, 기..."
1,0,0.997724,0,메인 시나리오 뼈대가 없고 보여주기라기엔 용과의 전투씬도 너무 부족함..더욱이 결말...
2,0,0.947795,1,올??? 찌질한 스커드가 노만 리더스 행님이었다니 !! ㅋㅋ
3,1,0.933018,0,5점이 적당하나 평점이 쓸데없이 높아서 낮출 필요가 있슴...
4,0,0.997567,0,"연출, 기획, 스토리 전부다 쓰레기..제인생 영화중에 제일 쓰레기.. 절대보지마요진심"
...,...,...,...,...
95,1,0.846441,1,"""""""철아 엄마가 꼭 니 신세 갚고 죽을께"""""""
96,0,0.997439,0,이건 뭐 .. 완전 삼류네
97,0,0.983951,0,으... 밑도 끝도 없음. 여자가 처음부터 자기혈청을 제공했으면 카오스가 안왔을거 아님?
98,0,0.997635,0,요즘에 보기에는 상당히 지루한 진부한 스릴러. 원초적 본능을 뛰어 넘기에도 힘든 영...


In [28]:
mask = preds_df['pred_label'] == preds_df['label']
len(preds_df[mask])

92

In [29]:
path = '/content/drive/MyDrive/Final_project/data/expanded_lotion_data.csv'
raw = pd.read_csv(path)
df = raw.copy()

In [32]:
reviews = df['reviews']

In [33]:
txts_td = pd.DataFrame(reviews)
txts_td.head()

,reviews
0,"순해서 너무 좋으나, 유분감이 느껴져서 적당히 바르는 게 좋을 것 같아요"
1,예전에 써봤을 때 보습력이 괜찮았던거 같아서 몇년만에 재구입해봤어요ㅋㅋ 여름에 무겁...
2,엄청 순하고 수분크림이지만 좀 되직해요. 가격도 저렵해서 좋아용
3,무난하게 사용하기 좋아요!! 적당히 촉촉하고 좋아요!!
4,순하고 좋아요 여름에 쓰기도 좋아요 끈적임이 없고 가격도 착해서 떨어지기전에 쟁여 두어요


In [34]:
txts_td.isnull().sum()

,0
reviews,0


In [36]:
txts_td['reviews'] = txts_td['reviews'].astype(str)

In [37]:
txts_td['reviews']

,reviews
0,"순해서 너무 좋으나, 유분감이 느껴져서 적당히 바르는 게 좋을 것 같아요"
1,예전에 써봤을 때 보습력이 괜찮았던거 같아서 몇년만에 재구입해봤어요ㅋㅋ 여름에 무겁...
2,엄청 순하고 수분크림이지만 좀 되직해요. 가격도 저렵해서 좋아용
3,무난하게 사용하기 좋아요!! 적당히 촉촉하고 좋아요!!
4,순하고 좋아요 여름에 쓰기도 좋아요 끈적임이 없고 가격도 착해서 떨어지기전에 쟁여 두어요
...,...
6747,셀퓨전씨…… 내가 너~~~~~무 좋아하는 브랜드 셀퓨전씨….. 썬크림은 말모말모 ...
6748,수분 부족형 지성인데 직원분 추천을 받고 샀습니다. 조금 건조하긴한데 촉촉한 앰플이...
6749,매번 피부 고민으로 기초제품 유목민이였는데 이 제품은 그래도 믿고 사용하는 제품입니...
6750,자극없이 사용가능하고 시원한 느낌이 들어 좋아요 다만 사용 후 당김이 있어요


In [38]:
# 정규화
import re
txts_td['reviews'].apply(lambda x : re.sub(r'[^ㄱ-ㅣ가-힣]+', ' ', x))

,reviews
0,순해서 너무 좋으나 유분감이 느껴져서 적당히 바르는 게 좋을 것 같아요
1,예전에 써봤을 때 보습력이 괜찮았던거 같아서 몇년만에 재구입해봤어요ㅋㅋ 여름에 무겁...
2,엄청 순하고 수분크림이지만 좀 되직해요 가격도 저렵해서 좋아용
3,무난하게 사용하기 좋아요 적당히 촉촉하고 좋아요
4,순하고 좋아요 여름에 쓰기도 좋아요 끈적임이 없고 가격도 착해서 떨어지기전에 쟁여 두어요
...,...
6747,셀퓨전씨 내가 너 무 좋아하는 브랜드 셀퓨전씨 썬크림은 말모말모 말해서 모해
6748,수분 부족형 지성인데 직원분 추천을 받고 샀습니다 조금 건조하긴한데 촉촉한 앰플이랑...
6749,매번 피부 고민으로 기초제품 유목민이였는데 이 제품은 그래도 믿고 사용하는 제품입니...
6750,자극없이 사용가능하고 시원한 느낌이 들어 좋아요 다만 사용 후 당김이 있어요


In [39]:
# 데이터프레임을 데이터셋으로 변환
from datasets import Dataset

dataset = Dataset.from_pandas(txts_td)

In [40]:
dataset

Dataset({
    features: ['reviews'],
    num_rows: 6752
})

In [41]:
# 모델 이름과 토크나이저 초기화
model_name = 'kykim/bert-kor-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [44]:
# 토크나이저 함수 정의
def tokenizer_func(examples):
    return tokenizer(
        examples['reviews'],
        padding='max_length',
        max_length=256,
        truncation=True
    )

In [45]:
tokenized_review_list = dataset.map(tokenizer_func, batched=True)

Map:   0%|          | 0/6752 [00:00<?, ? examples/s]

In [50]:
# 데이터셋 크기 확인
dataset_size = len(tokenized_review_list)
train_num_samples = min(10000, dataset_size)

In [51]:
txts_td = tokenized_review_list.select(range(train_num_samples))
txts_td

Dataset({
    features: ['reviews', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 6752
})

In [53]:
# 입력 텍스트를 512 토큰 이하로 자르기
def truncate_texts(texts, max_length=512):
    truncated_texts = []
    for text in texts:
        tokens = tokenizer.encode(text, add_special_tokens=True, max_length=max_length, truncation=True)
        truncated_text = tokenizer.decode(tokens, skip_special_tokens=True)
        truncated_texts.append(truncated_text)
    return truncated_texts

txts_td = truncate_texts(txts_td['reviews'])

# 예측 수행
preds_txts = pipe(txts_td)
preds_txts_df = pd.DataFrame(preds_txts)
preds_txts_df.info()
preds_txts_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6752 entries, 0 to 6751
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   6752 non-null   object 
 1   score   6752 non-null   float64
dtypes: float64(1), object(1)
memory usage: 105.6+ KB


,label,score
0,LABEL_1,0.902873
1,LABEL_1,0.978884
2,LABEL_1,0.958358
3,LABEL_1,0.925863
4,LABEL_1,0.966229


In [54]:
preds_txts_df['label'] = preds_txts_df['label'].map({'LABEL_1':1, 'LABEL_0':0})

In [55]:
review_labeling_df = pd.DataFrame(txts_td)
review_labeling_df.head()

,0
0,"순해서 너무 좋으나, 유분감이 느껴져서 적당히 바르는 게 좋을 것 같아요"
1,예전에 써봤을 때 보습력이 괜찮았던거 같아서 몇년만에 재구입해봤어요ㅋㅋ 여름에 무겁...
2,엄청 순하고 수분크림이지만 좀 되직해요. 가격도 저렵해서 좋아용
3,무난하게 사용하기 좋아요!! 적당히 촉촉하고 좋아요!!
4,순하고 좋아요 여름에 쓰기도 좋아요 끈적임이 없고 가격도 착해서 떨어지기전에 쟁여 두어요


In [60]:
review_labeling_df.columns = ['reviews']
review_labeling_df.head()

,reviews
0,"순해서 너무 좋으나, 유분감이 느껴져서 적당히 바르는 게 좋을 것 같아요"
1,예전에 써봤을 때 보습력이 괜찮았던거 같아서 몇년만에 재구입해봤어요ㅋㅋ 여름에 무겁...
2,엄청 순하고 수분크림이지만 좀 되직해요. 가격도 저렵해서 좋아용
3,무난하게 사용하기 좋아요!! 적당히 촉촉하고 좋아요!!
4,순하고 좋아요 여름에 쓰기도 좋아요 끈적임이 없고 가격도 착해서 떨어지기전에 쟁여 두어요


In [61]:
preds_txts_df = pd.concat([preds_txts_df, review_labeling_df], axis=1)
preds_txts_df

,label,score,0,reviews
0,1,0.902873,"순해서 너무 좋으나, 유분감이 느껴져서 적당히 바르는 게 좋을 것 같아요","순해서 너무 좋으나, 유분감이 느껴져서 적당히 바르는 게 좋을 것 같아요"
1,1,0.978884,예전에 써봤을 때 보습력이 괜찮았던거 같아서 몇년만에 재구입해봤어요ㅋㅋ 여름에 무겁...,예전에 써봤을 때 보습력이 괜찮았던거 같아서 몇년만에 재구입해봤어요ㅋㅋ 여름에 무겁...
2,1,0.958358,엄청 순하고 수분크림이지만 좀 되직해요. 가격도 저렵해서 좋아용,엄청 순하고 수분크림이지만 좀 되직해요. 가격도 저렵해서 좋아용
3,1,0.925863,무난하게 사용하기 좋아요!! 적당히 촉촉하고 좋아요!!,무난하게 사용하기 좋아요!! 적당히 촉촉하고 좋아요!!
4,1,0.966229,순하고 좋아요 여름에 쓰기도 좋아요 끈적임이 없고 가격도 착해서 떨어지기전에 쟁여 두어요,순하고 좋아요 여름에 쓰기도 좋아요 끈적임이 없고 가격도 착해서 떨어지기전에 쟁여 두어요
...,...,...,...,...
6747,1,0.982504,셀퓨전씨 내가 너 ~ ~ ~ ~ ~ 무 좋아하는 브랜드 셀퓨전씨.. 썬크림은 말모말...,셀퓨전씨 내가 너 ~ ~ ~ ~ ~ 무 좋아하는 브랜드 셀퓨전씨.. 썬크림은 말모말...
6748,1,0.925493,수분 부족형 지성인데 직원분 추천을 받고 샀습니다. 조금 건조하긴한데 촉촉한 앰플이...,수분 부족형 지성인데 직원분 추천을 받고 샀습니다. 조금 건조하긴한데 촉촉한 앰플이...
6749,1,0.971142,매번 피부 고민으로 기초제품 유목민이였는데 이 제품은 그래도 믿고 사용하는 제품입니...,매번 피부 고민으로 기초제품 유목민이였는데 이 제품은 그래도 믿고 사용하는 제품입니...
6750,1,0.941167,자극없이 사용가능하고 시원한 느낌이 들어 좋아요 다만 사용 후 당김이 있어요,자극없이 사용가능하고 시원한 느낌이 들어 좋아요 다만 사용 후 당김이 있어요


In [64]:
preds_txts_df.drop([0], axis=1, inplace=True)
preds_txts_df.head()

,label,score,reviews
0,1,0.902873,"순해서 너무 좋으나, 유분감이 느껴져서 적당히 바르는 게 좋을 것 같아요"
1,1,0.978884,예전에 써봤을 때 보습력이 괜찮았던거 같아서 몇년만에 재구입해봤어요ㅋㅋ 여름에 무겁...
2,1,0.958358,엄청 순하고 수분크림이지만 좀 되직해요. 가격도 저렵해서 좋아용
3,1,0.925863,무난하게 사용하기 좋아요!! 적당히 촉촉하고 좋아요!!
4,1,0.966229,순하고 좋아요 여름에 쓰기도 좋아요 끈적임이 없고 가격도 착해서 떨어지기전에 쟁여 두어요


In [66]:
# 같은 행에서 df['label']의 값이 1이면 df['review_list']에 [긍정]이라는 텍스트를 추가
preds_txts_df.loc[preds_txts_df['label'] == 1, 'reviews'] = '[긍정] ' + preds_txts_df['reviews']

# 같은 행에서 df['label']의 값이 0이면 df['review_list']에 [부정]이라는 텍스트를 추가
preds_txts_df.loc[preds_txts_df['label'] == 0, 'reviews'] = '[부정] ' + preds_txts_df['reviews']

preds_txts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6752 entries, 0 to 6751
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   label    6752 non-null   int64  
 1   score    6752 non-null   float64
 2   reviews  6752 non-null   object 
dtypes: float64(1), int64(1), object(1)
memory usage: 158.4+ KB


In [67]:
preds_txts_df.reset_index(drop=True)

,label,score,reviews
0,1,0.902873,"[긍정] 순해서 너무 좋으나, 유분감이 느껴져서 적당히 바르는 게 좋을 것 같아요"
1,1,0.978884,[긍정] 예전에 써봤을 때 보습력이 괜찮았던거 같아서 몇년만에 재구입해봤어요ㅋㅋ 여...
2,1,0.958358,[긍정] 엄청 순하고 수분크림이지만 좀 되직해요. 가격도 저렵해서 좋아용
3,1,0.925863,[긍정] 무난하게 사용하기 좋아요!! 적당히 촉촉하고 좋아요!!
4,1,0.966229,[긍정] 순하고 좋아요 여름에 쓰기도 좋아요 끈적임이 없고 가격도 착해서 떨어지기전...
...,...,...,...
6747,1,0.982504,[긍정] 셀퓨전씨 내가 너 ~ ~ ~ ~ ~ 무 좋아하는 브랜드 셀퓨전씨.. 썬크림...
6748,1,0.925493,[긍정] 수분 부족형 지성인데 직원분 추천을 받고 샀습니다. 조금 건조하긴한데 촉촉...
6749,1,0.971142,[긍정] 매번 피부 고민으로 기초제품 유목민이였는데 이 제품은 그래도 믿고 사용하는...
6750,1,0.941167,[긍정] 자극없이 사용가능하고 시원한 느낌이 들어 좋아요 다만 사용 후 당김이 있어요


In [69]:
preds_txts_df.to_csv('/content/drive/MyDrive/Final_project/data/sentiment_lotion.csv')